# Lesson 5 - Creating an A2A Sequential Chain Agent with ADK

Now that you have two active agents (Policy Agent and Research Agent), you will orchestrate them. In this lesson, you will use Google ADK's `SequentialAgent` to create a workflow where a user's query is processed by the Research Agent first, and then the Policy Agent, or vice versa, based on the sequential definition. You will use `RemoteA2aAgent` to connect to the servers you started in previous lessons.

```mermaid
graph LR
    classDef orchestrator fill:#f9f,stroke:#333,stroke-width:2px;
    classDef agent fill:#e1f5fe,stroke:#0277bd,stroke-width:2px;
    classDef tool fill:#fff3e0,stroke:#ef6c00,stroke-width:1px,stroke-dasharray: 5 5;

    subgraph Sequential [Sequential Agent]
        direction LR
        
        HealthcareAgent["<b>Healthcare Research<br/>Agent</b>"]
        PolicyAgent["<b>Policy<br/>Agent</b>"]
        
        HealthcareAgent --> PolicyAgent
    end

    class HealthcareAgent,PolicyAgent agent;
```

## 5.1. Start the A2A Servers

Ensure both the Policy Agent (Terminal 1) and Research Agent (Terminal 2) are running.
- Open two terminals as instructed below.
- In Terminal 1, type: `uv run a2a_policy_agent.py`
- In Terminal 2, type: `uv run a2a_research_agent.py`

<div style="background-color:#e8f0fe; padding:15px; border-left:5px solid #4285f4; border-radius:4px">
    <b>Terminal Access:</b> Please open two new terminal windows in your Jupyter environment to run the servers.
    <br>You can typically do this by selecting <i>File -> New -> Terminal</i> from the menu.
</div>

## 5.2. Define the Sequential Workflow

Here you will:
1.  Define two `RemoteA2aAgent` instances. These act as proxies that know how to communicate with your running servers via A2A.
2.  Create a `SequentialAgent` named `root_agent`. This agent contains the logic to route the workflow through the sub-agents.
3.  Use an `InMemoryRunner` to execute the flow with a specific prompt.


In [12]:
import os

from helpers import setup_env

setup_env()

from IPython.display import Markdown, display
from google.adk.agents import SequentialAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
)
from google.adk.runners import InMemoryRunner

In [13]:
host = os.getenv("AGENT_HOST")
policy_port = os.getenv("POLICY_AGENT_PORT")
research_port = os.getenv("RESEARCH_AGENT_PORT")

In [14]:
policy_agent = RemoteA2aAgent(
    name="policy_agent",
    agent_card=f"http://{host}:{policy_port}",
)
print("\tℹ️", f"{policy_agent.name} initialized")

	ℹ️ policy_agent initialized


In [15]:
health_research_agent = RemoteA2aAgent(
    name="health_research_agent",
    agent_card=f"http://{host}:{research_port}",
)
print("\tℹ️", f"{health_research_agent.name} initialized")

	ℹ️ health_research_agent initialized


In [16]:
root_agent = SequentialAgent(
    name="root_agent",
    description="Healthcare Routing Agent",
    sub_agents=[
        health_research_agent,
        policy_agent,
    ],
)
print("\tℹ️", f"{root_agent.name} initialized")

	ℹ️ root_agent initialized


## 5.3. Run the Sequential Chain

The `InMemoryRunner` executes the agent. The prompt will trigger the Research Agent to find general info, and then (sequentially) the Policy Agent to check coverage details.


In [17]:
prompt = "How can I get mental health therapy?"

In [18]:
print("Running Healthcare Workflow Agent")

runner = InMemoryRunner(root_agent)

for event in await runner.run_debug(prompt, quiet=True):
    if event.is_final_response() and event.content:
        display(Markdown(event.content.parts[0].text))

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/loinguyen/A2A/A2A_learning/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Running Healthcare Workflow Agent



On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-3-pro\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-3-pro\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-3-pro\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-3-pro\nPlease retry in 46.17999665s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-3-pro'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-3-pro', 'location': 'global'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerMinute-FreeTier', 'quotaDimensions': {'model': 'gemini-3-pro', 'location': 'global'}}, {'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_input_token_count', 'quotaId': 'GenerateContentInputTokensPerModelPerDay-FreeTier', 'quotaDimensions': {'model': 'gemini-3-pro', 'location': 'global'}}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '46s'}]}}

Under the Anthem Blue Cross gHIP HDHP plan, you can access mental health therapy through outpatient services. Here are the details on how to get care and what it will cost:

### **1. Finding a Provider**
You do not need a referral to see a specialist, including mental health professionals. To find a provider and ensure you pay the lowest rate, you can:
*   Visit **includedhealth.com/google**
*   Call **(855) 431-5540** for a list of network providers.

The network you use depends on your location:
*   **NY:** NY Blue Access PPO
*   **GA:** GA Blue Open Access POS
*   **Other states:** Blue Card PPO

### **2. Cost of Therapy (Outpatient Services)**
Costs for mental health services are subject to your **overall deductible** (\$1,700 for individuals / \$3,400 for families for in-network care). Once the deductible is met:

*   **In-Network Office Visit:** You pay **10% coinsurance**.
*   **Out-of-Network Office Visit:** You pay **30% coinsurance**.

### **3. Inpatient Services**
If you require inpatient mental health or substance abuse services, the cost sharing is:
*   **In-Network:** 10% coinsurance (after deductible).
*   **Out-of-Network:** 30% coinsurance (after deductible).

For more detailed information or to view the complete terms of coverage, you can visit **go/benefitdocuments**.

## 5.4. Resources

- [Google ADK Sequential Agents](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)